In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/ProtonX_Deep_Learning/Transfer_Learning

/content/drive/MyDrive/ProtonX_Deep_Learning/Transfer_Learning


In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O ./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2021-05-27 16:17:01--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.137.128, 142.250.141.128, 2607:f8b0:4023:c03::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.137.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

./inception_v3_weig 100%[===================>]  83.84M  67.1MB/s    in 1.2s    

2021-05-27 16:17:02 (67.1 MB/s) - ‘./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model

from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(input_shape = (150, 150, 3),
                                include_top = False,
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

In [ ]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 74, 74, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 74, 74, 32)   96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 74, 74, 32)   0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [ ]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [ ]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 74, 74, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 74, 74, 32)   96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 74, 74, 32)   0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')

In [ ]:
last_output = last_layer.output

In [ ]:
last_output

<KerasTensor: shape=(None, 7, 7, 768) dtype=float32 (created by layer 'mixed7')>

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)

model = Model( pre_trained_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001),
              loss = 'binary_crossentropy',
              metrics = ['acc'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
train_folder = './data/cats_and_dogs_filtered/train'
valid_folder = './data/cats_and_dogs_filtered/validation'

In [ ]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O ./cats_and_dogs_filtered.zip

--2021-05-27 16:17:04--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 2607:f8b0:4023:c0b::80
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  56.0MB/s    in 1.2s    

2021-05-27 16:17:06 (56.0 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
main_folder = './data'
!unzip -q cats_and_dogs_filtered.zip -d $main_folder

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
validation_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_folder,
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (150, 150))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  validation_datagen.flow_from_directory( valid_folder,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary',
                                                          target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'cats': 0, 'dogs': 1}

In [ ]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 20,
            validation_steps = 50,
            verbose = 2)
model.save("./data/transferLearningDogCat.h5")

Epoch 1/20
100/100 - 24s - loss: 0.3677 - acc: 0.8535 - val_loss: 0.2152 - val_acc: 0.9140
Epoch 2/20
100/100 - 21s - loss: 0.2198 - acc: 0.9135 - val_loss: 0.1245 - val_acc: 0.9530
Epoch 3/20
100/100 - 20s - loss: 0.2036 - acc: 0.9195 - val_loss: 0.3294 - val_acc: 0.9080
Epoch 4/20
100/100 - 20s - loss: 0.1977 - acc: 0.9240 - val_loss: 0.0815 - val_acc: 0.9660
Epoch 5/20
100/100 - 20s - loss: 0.1893 - acc: 0.9330 - val_loss: 0.1236 - val_acc: 0.9580
Epoch 6/20
100/100 - 20s - loss: 0.1776 - acc: 0.9360 - val_loss: 0.1407 - val_acc: 0.9580
Epoch 7/20
100/100 - 19s - loss: 0.1698 - acc: 0.9405 - val_loss: 0.1236 - val_acc: 0.9680
Epoch 8/20
100/100 - 19s - loss: 0.1432 - acc: 0.9515 - val_loss: 0.0954 - val_acc: 0.9670
Epoch 9/20
100/100 - 19s - loss: 0.1802 - acc: 0.9460 - val_loss: 0.0994 - val_acc: 0.9690
Epoch 10/20
100/100 - 20s - loss: 0.1559 - acc: 0.9460 - val_loss: 0.1463 - val_acc: 0.9620
Epoch 11/20
100/100 - 19s - loss: 0.1600 - acc: 0.9440 - val_loss: 0.0924 - val_acc: 0.97

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import tensorflow as tf

uploaded = files.upload()

for fn in uploaded.keys():

  # prediction Process

  path = './' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img) / 255

  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  print(images.shape)
  saved_model = tf.keras.models.load_model("/content/drive/MyDrive/ProtonX_Deep_Learning/Transfer_Learning/data/transferLearningDogCat.h5")

  classes = model.predict(images, batch_size=10)

  if classes[0] > 0.5:
    print(fn + " is a dog")
  else:
    print(fn + " is a cat")

Saving photo-1-15222954332811717937809.jpg to photo-1-15222954332811717937809.jpg
(1, 150, 150, 3)
photo-1-15222954332811717937809.jpg is a dog
